In [1]:
!pip install openai==0.28
!pip install -U datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 2.1 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.81.0
    Uninstalling openai-1.81.0:
      Successfully uninstalled openai-1.81.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 14.3 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
  Attempting uninstall: datasets
    Found existing installation: datasets 2.14.4
    Uninstalling datasets-2.14.4:
      Successfully uninstalled datasets-2.14.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2025.3.0 which is incompatible.
torch 2.6.0+c

In [2]:
import re
import pandas as pd
import openai
from datasets import load_dataset
from huggingface_hub import login

In [3]:
login()

In [5]:
openai.api_key = 'Your-api'
# comment the below line if you use original OPENAI api
openai.api_base = 'https://api.avalai.ir/v1'

In [7]:
prompt = f'''
یک پرسش پزشکی چند گزینه ای انگلیسی به شما داده شده است وظیفه شما این است که این پرسش را به فارسی ترجمه کنید لطفا در فرمت زیر و بدون ارائه توضیحات اضافی پاسخ دهید.
# question
Question: {{question}}
Option A: {{optiona}}
Option B: {{optionb}}
Option C: {{optionc}}
Option D: {{optiond}}
Expression: {{expression}}

# format
Question:
Option A:
Option B:
Option C:
Option D:
Expression:
'''

In [8]:
def get_qa_elements(text):
    text = text + '\n'
    pattern1 = r"Question: (.*?)\nOption A: (.*?)\nOption B: (.*?)\nOption C: (.*?)\nOption D: (.*?)\nExpression: (.*?)\n"
    pattern2 = r"question: (.*?)\noption a: (.*?)\noption b: (.*?)\noption c: (.*?)\noption d: (.*?)\nexpression: (.*?)\n"
    match1 = re.search(pattern1, text, re.DOTALL | re.IGNORECASE)
    match2 = re.search(pattern2, text, re.DOTALL | re.IGNORECASE)
    if match1:
       question = match1.group(1).strip()
       option_a = match1.group(2).strip()
       option_b = match1.group(3).strip()
       option_c = match1.group(4).strip()
       option_d = match1.group(5).strip()
       expression = match1.group(6).strip()
    elif match2:
       question = match2.group(1).strip()
       option_a = match2.group(2).strip()
       option_b = match2.group(3).strip()
       option_c = match2.group(4).strip()
       option_d = match2.group(5).strip()
       expression = match2.group(6).strip()
    else:
       print('not matched input manually...  ' + '\n')
       print(text + '\n')
       question = input('question: ')
       print()
       option_a = input('option_a: ')
       print()
       option_b = input('option_b: ')
       print()
       option_c = input('option_c: ')
       print()
       option_d = input('option_d: ')
       print()
       expression = input('expression: ')
       print()
       print('$$$$$$$$$')
    return question,option_a,option_b,option_c,option_d,expression

def append_record_to_excel(file_path,id,question,op1,op2,op3,op4,expression,
                           correct_option,choice_type,subject_name,
                           topic_name):
    if question == '.':
       pass
    else:
        new_record = {
            'id': id,
            'Question': question,
            'Option1': op1,
            'Option2': op2,
            'Option3': op3,
            'Option4': op4,
            'Expression': expression,
            'Correct_answer': correct_option,
            'choice_type':choice_type,
            'subject_name':subject_name,
            'topic_name':topic_name
        }
        new_record_df = pd.DataFrame([new_record])
        try:
            existing_df = pd.read_excel(file_path)
            updated_df = pd.concat([existing_df, new_record_df], ignore_index=True)
        except FileNotFoundError:
            updated_df = new_record_df

        updated_df.to_excel(file_path, index=False)

def get_deepseek_answer(prompt):
    response = openai.ChatCompletion.create(
        model="deepseek-chat",
        messages=[
            {"role": "user", "content": prompt}
        ],
        temperature = 0.0
    )
    result = response['choices'][0]['message']['content'].strip().lower()
    return result

In [9]:
dataset = load_dataset('mehrdadgh/medmcqa', split='train')

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/727 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/85.5M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/182822 [00:00<?, ? examples/s]

In [ ]:
i = 0
bgn = 2000
for j in range(182822):
        record = dataset[j]
        if i <= bgn - 1:
           i += 1
           continue
        id = record['id']
        question = record['question']
        option1 = record['opa']
        option2 = record['opb']
        option3 = record['opc']
        option4 = record['opd']
        cop = record['cop']
        choice_type = record['choice_type']
        exp = record['exp']
        subject_name = record['subject_name']
        topic_name = record['topic_name']
        translated_qa = get_deepseek_answer(prompt.format(question=question,optiona=option1,optionb=option2,optionc=option3,optiond=option4,expression=exp))
        gquestion,goption_a,goption_b,goption_c,goption_d,gexpression = get_qa_elements(translated_qa)
        append_record_to_excel(file_path='./translated_qas.xlsx',id=id,question=gquestion,
                               op1=goption_a,op2=goption_b,op3=goption_c,op4=goption_d,
                               expression=gexpression,correct_option=cop,choice_type=choice_type,
                               subject_name=subject_name,topic_name=topic_name)
        i += 1
        print(i)
        print('==================================================================================')

2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023
2024
2025
2026
2027
2028
2029
2030
2031
2032
2033
2034
2035
2036
2037
2038
2039
2040
2041
2042
2043
2044
2045
2046
2047
2048
2049
2050
2051
2052
2053
2054
2055
2056
2057
2058
2059
2060
2061
2062
2063
2064
2065
2066
2067
2068
2069
2070
2071
2072
2073
2074
2075
2076
2077
2078
2079
2080
2081
2082
2083
2084
2085
2086
2087
2088
2089
2090
2091
2092
2093
2094
2095
2096
2097
2098
2099
2100
2101
2102
2103
2104
2105
2106
2107
2108
2109
2110
2111
2112
2113
2114
2115
2116
2117
2118
2119
2120
2121
2122
2123
2124
2125
2126
2127
2128
2129
2130
2131
2132
2133
2134
2135
2136
2137
2138
2139
2140
2141
2142
2143
2144
2145
2146
2147
2148
2149
2150
2151
2152
2153
2154
2155
2156
2157
2158
2159
2160
2161
2162
2163
2164
2165
2166
2167
2168
2169
2170
2171
2172
2173
2174
2175
2176
2177
2178
2179
2180
2181
2182
2183
2184
2185
2186
2187
2188
2189
2190
2191
2192
2193
2194
2195
2196
2197
2198
2199
2200
